In [145]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import numpy as np
import matplotlib.pyplot as plt
from string import punctuation

DICTIONARYSIZE = 300
MIDDLELAYERSIZE = DICTIONARYSIZE//20
SKIPWINDOW = 2
DATATEXT = 'ham.txt'

In [146]:
model = Sequential()
model.add(Dense(input_dim=DICTIONARYSIZE, units = MIDDLELAYERSIZE, activation = "relu"))
model.add(Dense(units = DICTIONARYSIZE, activation = "softmax"))

sgd = SGD()
model.compile(loss="categorical_crossentropy",
             optimizer = sgd,
             metrics = ['accuracy'])

In [149]:
#Clean data set
def clean(lines):
    ret = []
    count = {}
    for line in lines:
        line = line.strip()
        split = line.split()
        if(len(split) == 0): continue
        #These lines are specific to the Hamlet data set. Add code for lines to disregard
        if(split[0].isupper()): continue
        if(split[0] == "enter"): continue
        for word in split:
            word = word.lower()
            if word[-1] in punctuation:
                word = word[:-1]
            if len(word) == 0: continue
            if word[0] in punctuation:
                word = word[1:]
            if len(word) == 0: continue
            ret.append(word)
            if word in count.keys():
                count[word] += 1
            else:
                count[word] = 1
            
    return ret, count

lines = open(DATATEXT,'r').readlines()
wordList, count = clean(lines)

indexToWord = list(set(wordList))
print(indexToWord[:10])

#Construct word-to-index dictionary
wordToIndex = {indexToWord[i]:i for i in range(len(indexToWord))}

['tripping', 'bladder', 'hammer', 'till', 'buds', 'linen', 'ransom', 'twice', 'scold', 'distract']


In [150]:
#(Duplication test)
for i in range(-100,-1):
    if (i%len(indexToWord)) != wordToIndex[indexToWord[i]]:
        print(i)

In [151]:
#Construct word-to-vector (one-hot) and vector-to-word (one-hot) dictionary
#The "hot" one will be indexed by popularity
wordToVector = {}
zeroVector = [0 for _ in range(DICTIONARYSIZE)]
#First sort word-to-index by word popularity
wordsByPopularityDoubles = sorted(count.items(), key = lambda x:x[1], reverse = True)
wordsByPopularity = [wordsByPopularityDoubles[i][0] for i in range(len(wordsByPopularityDoubles))]

for i in range(DICTIONARYSIZE):
    vec = zeroVector.copy()
    vec[i] = 1
    wordToVector[wordsByPopularity[i]] = vec
    
wordsByPopularityDoubles[DICTIONARYSIZE]

('four', 38)

In [152]:
print("Most popular words:\n" + str(wordsByPopularity[:19]))
print(str(len(wordsByPopularity)) + " words total.")

Most popular words:
['the', 'and', 'of', 'to', 'a', 'i', 'you', 'are', 'my', 'in', 'that', 'coming', 'british', 'is', 'it', 'with', 'not', 'this', 'me']
9615 words total.


In [153]:
#Create training arrays (Skip-gram)
#Currently creates the same batch every call
#May be stochastic in the future
def generate_batch():
    x_train = []
    y_train = []
    #Start the moving window. Windex is the index of the center of the window.
    #OPTIMIZE: Use np arrays
    #OPTIMIZE: Implement mini-batch features shown in the word2vec tutorial
    for windex in range(SKIPWINDOW, len(wordList) - SKIPWINDOW):
        centerWord = wordList[windex]
        #OPTIMIZE replace this lookup using a nice dictionary OOH OR A NUMPY ARRAY BOOLEAN THING
        if not (centerWord in wordToVector.keys()): continue
        for contex in range(windex-SKIPWINDOW-1, windex+SKIPWINDOW):
            contextWord = wordList[contex]
            #Skip the center word (it is a skip-gram after all...)
            if contex == windex: continue
            #OPTIMIZE replace this lookup using a nice dictionary
            if not (contextWord in wordToVector.keys()): continue
            x_train.append(wordToVector[wordList[windex]])
            y_train.append(wordToVector[wordList[contex]])
    return np.array(x_train), np.array(y_train)

In [154]:
x_train, y_train = generate_batch()

In [159]:
print("Vectors: " + str(len(x_train)))
model.fit(x_train, y_train, epochs = 100, batch_size = 32)

Vectors: 190372
Epoch 1/100
190372/190372 [==============================] - 63s 330us/step - loss: 4.7366 - acc: 0.0788
Epoch 2/100
190372/190372 [==============================] - 70s 366us/step - loss: 4.6870 - acc: 0.0901
Epoch 3/100
190372/190372 [==============================] - 58s 305us/step - loss: 4.6703 - acc: 0.0923
Epoch 4/100
190372/190372 [==============================] - 58s 306us/step - loss: 4.6619 - acc: 0.0936
Epoch 5/100
190372/190372 [==============================] - 58s 306us/step - loss: 4.6567 - acc: 0.0942
Epoch 6/100
190372/190372 [==============================] - 56s 297us/step - loss: 4.6529 - acc: 0.0946
Epoch 7/100
190372/190372 [==============================] - 60s 317us/step - loss: 4.6499 - acc: 0.0950
Epoch 8/100
190372/190372 [==============================] - 59s 312us/step - loss: 4.6471 - acc: 0.0957
Epoch 9/100
190372/190372 [==============================] - 60s 313us/step - loss: 4.6443 - acc: 0.0961
Epoch 10/100
190372/190372 [===========

190372/190372 [==============================] - 62s 323us/step - loss: 4.5274 - acc: 0.1117
Epoch 78/100
190372/190372 [==============================] - 61s 320us/step - loss: 4.5267 - acc: 0.1114
Epoch 79/100
190372/190372 [==============================] - 61s 319us/step - loss: 4.5258 - acc: 0.1116
Epoch 80/100
190372/190372 [==============================] - 61s 319us/step - loss: 4.5251 - acc: 0.1116
Epoch 81/100
190372/190372 [==============================] - 61s 320us/step - loss: 4.5242 - acc: 0.1117
Epoch 82/100
190372/190372 [==============================] - 61s 320us/step - loss: 4.5234 - acc: 0.1118
Epoch 83/100
190372/190372 [==============================] - 62s 328us/step - loss: 4.5226 - acc: 0.1119
Epoch 84/100
190372/190372 [==============================] - 59s 312us/step - loss: 4.5219 - acc: 0.1120
Epoch 85/100
190372/190372 [==============================] - 60s 313us/step - loss: 4.5211 - acc: 0.1122
Epoch 86/100
190372/190372 [==============================]

In [163]:
for i in range(DICTIONARYSIZE):
    a = model.predict(np.array([wordToVector[wordsByPopularity[i]]]))[0]
    printConfidence(a)

1.17%: the
1.14%: are
1.13%: coming
1.11%: british
1.05%: and
1.04%: to
1.03%: of
1.02%: you
1.03%: my
1.01%: it
1.01%: i
1.02%: that
1.02%: is
1.01%: with
1.0%: me
1.01%: be
1.0%: him
1.0%: so
1.02%: have
1.01%: we
1.0%: no
1.01%: do
1.0%: all
1.0%: shall
1.0%: thou
1.0%: hamlet
1.01%: there
1.0%: sir
1.0%: go
1.0%: well
1.0%: then
1.01%: which
1.01%: such
1.0%: upon
1.0%: thee
1.01%: say
1.2%: the
1.12%: are
1.12%: coming
1.13%: british
1.03%: and
1.02%: to
1.02%: of
1.02%: you
1.0%: my
1.01%: a
1.01%: it
1.02%: his
1.02%: not
1.0%: with
1.01%: me
1.02%: he
1.02%: lord
1.0%: will
1.01%: we
1.0%: no
1.01%: on
1.01%: come
1.0%: most
1.0%: there
1.0%: first
1.0%: then
1.0%: which
1.0%: very
1.0%: give
1.0%: some
1.0%: here
1.0%: say
1.01%: much
1.01%: father
1.16%: the
1.13%: are
1.12%: coming
1.11%: british
1.07%: and
1.05%: to
1.05%: of
1.03%: you
1.03%: my
1.02%: that
1.01%: is
1.01%: his
1.02%: with
1.02%: me
1.03%: he
1.01%: be
1.01%: lord
1.0%: so
1.04%: have
1.05%: we
1.03%: no
1

1.06%: there
1.01%: sir
1.03%: go
1.02%: well
1.02%: us
1.02%: first
1.06%: then
1.0%: know
1.03%: enter
1.04%: hath
1.06%: which
1.04%: such
1.0%: here
1.03%: upon
1.06%: say
1.04%: father
1.19%: the
1.14%: are
1.13%: coming
1.12%: british
1.04%: and
1.03%: to
1.02%: of
1.02%: you
1.02%: my
1.0%: a
1.01%: it
1.01%: i
1.01%: that
1.01%: is
1.0%: not
1.0%: with
1.0%: be
1.0%: have
1.15%: the
1.15%: are
1.13%: coming
1.1%: british
1.08%: and
1.07%: to
1.05%: of
1.03%: you
1.05%: my
1.01%: it
1.02%: i
1.04%: that
1.04%: is
1.02%: with
1.0%: me
1.0%: he
1.01%: be
1.01%: him
1.01%: so
1.04%: have
1.03%: we
1.02%: no
1.02%: do
1.0%: on
1.01%: our
1.01%: all
1.01%: by
1.02%: shall
1.0%: come
1.02%: thou
1.01%: from
1.02%: hamlet
1.02%: there
1.02%: sir
1.02%: go
1.02%: well
1.0%: us
1.02%: then
1.01%: enter
1.01%: hath
1.02%: which
1.0%: why
1.02%: such
1.02%: upon
1.02%: thee
1.03%: say
1.18%: the
1.14%: are
1.13%: coming
1.12%: british
1.04%: and
1.03%: to
1.02%: of
1.02%: you
1.02%: my
1.0

1.02%: that
1.02%: is
1.01%: with
1.0%: me
1.01%: be
1.0%: him
1.0%: so
1.02%: have
1.01%: we
1.0%: no
1.01%: do
1.0%: all
1.0%: shall
1.0%: thou
1.01%: hamlet
1.01%: there
1.0%: sir
1.01%: go
1.01%: well
1.01%: then
1.01%: which
1.01%: such
1.0%: upon
1.0%: thee
1.01%: say
1.21%: the
1.09%: are
1.11%: coming
1.14%: british
1.03%: and
1.03%: of
1.02%: you
1.01%: a
1.05%: his
1.03%: not
1.0%: with
1.03%: me
1.05%: he
1.05%: lord
1.0%: what
1.02%: will
1.03%: we
1.02%: no
1.05%: on
1.03%: come
1.0%: how
1.01%: hamlet
1.03%: most
1.03%: there
1.02%: them
1.0%: us
1.03%: first
1.03%: then
1.01%: know
1.01%: enter
1.02%: hath
1.03%: which
1.03%: very
1.03%: give
1.01%: such
1.02%: some
1.04%: here
1.03%: say
1.0%: man
1.04%: much
1.05%: father
1.0%: clown
1.19%: the
1.13%: are
1.13%: coming
1.13%: british
1.03%: and
1.03%: to
1.01%: of
1.02%: you
1.02%: my
1.01%: a
1.01%: in
1.01%: it
1.01%: i
1.01%: that
1.01%: is
1.0%: his
1.01%: not
1.0%: this
1.0%: but
1.0%: lord
1.21%: the
1.1%: are
1.

In [160]:
a = model.predict(np.array([wordToVector["hamlet"]]))[0]
printConfidence(a)
print(a)

NameError: name 'printConfidence' is not defined

In [161]:
def vectorToWord(vector):
    for i in range(len(vector)):
        if(vector[i]==1):
            return wordsByPopularity[i]

In [162]:
def printConfidence(prediction):
    for i,confidence in enumerate(prediction):
        if confidence > 0.01:
            print(str(round((confidence * 100), 2)) + "%: " + wordsByPopularity[i])

In [197]:
#What is wrong with my training set?
print("y_train|x_train")
for i in range(len(y_train)):
    print(vectorToWord(y_train[i]) + "|" + vectorToWord(x_train[i]))

y_train|x_train
there|me
me|and
king|the
the|king
the|he
king|he
you|he
king|you
he|you
come|you
he|come
you|come
most|come
he|most
you|most
come|most
come|upon
most|upon
your|upon
most|your
upon|your
upon|tis
your|tis
now|tis
your|now
tis|now
to|thee
thee|to
to|for
this|for
for|this
for|much
this|much
much|tis
tis|and
i|and
and|i
i|at
at|have
you|have
at|you
have|you
a|not
not|a
a|well
good|well
well|good
well|if
good|if
you|if
good|you
if|you
do|you
if|do
you|do
do|and
and|the
the|of
my|of
the|my
of|my
my|them
make|them
them|make
them|enter
make|enter
enter|and
and|to
this|to
to|this
to|and
this|and
and|to
the|to
and|the
to|the
to|give
the|give
you|give
the|you
give|you
good|you
give|good
you|good
good|hath
hath|you
you|my
my|give
you|give
my|you
give|you
good|you
give|good
you|good
what|say
say|what
is|what
say|is
what|is
what|there
is|there
good|what
what|this
but|tis
tis|but
our|but
tis|our
but|our
but|and
our|and
will|and
our|will
and|will
not|will
and|not
will|not
let|not
and|le

then|like
it|like
a|like
it|a
like|a
a|upon
a|upon
upon|a
a|i
have|i
i|have
i|the
have|the
the|that
is|that
the|is
that|is
the|is
that|the
is|the
is|to
the|to
the|to
the|the
to|the
the|with
his|with
with|his
with|and
his|and
the|of
of|and
at|and
of|at
and|at
his|at
and|his
at|his
his|in
in|or
or|in
in|or
or|the
the|and
his|to
to|his
to|and
his|and
of|and
his|of
and|of
the|of
and|the
of|the
the|this
it|on
the|on
it|the
on|the
on|of
the|of
the|of
the|the
of|the
of|some
the|some
say|some
the|say
some|say
that|say
some|that
say|that
that|that
our|is
is|the
the|of
of|all
all|and
then|and
and|then
they|then
and|they
then|they
say|they
and|say
then|say
they|say
no|say
then|no
they|no
say|no
the|are
are|then
no|then
are|no
then|no
no|no
hath|to
to|so
so|and
so|and
so|so
and|so
and|is
so|is
the|is
so|the
is|the
is|so
the|so
have|so
the|have
so|have
i|have
so|i
have|i
have|and
i|and
do|and
i|do
and|do
in|do
and|in
do|in
in|it
but|it
it|but
it|the
but|the
the|in
the|of
our|we
we|our
our|and
by|an

we|and
our|we
and|we
and|you
we|you
you|you
to|you
you|to
you|to
you|here
to|here
in|here
to|in
here|in
the|in
here|the
in|the
in|and
the|and
and|of
our|of
and|our
of|our
of|our
our|our
our|and
and|our
and|let
our|let
not|let
our|not
let|not
thy|not
let|thy
not|thy
thy|her
her|hamlet
why|hamlet
her|why
hamlet|why
tis|why
hamlet|tis
why|tis
a|tis
hamlet|a
why|a
tis|a
tis|and
a|and
a|and
a|a
and|a
a|be
as|be
be|as
be|in
as|in
in|come
this|come
come|this
come|and
this|and
and|of
hamlet|of
of|hamlet
hamlet|to
my|to
to|my
to|in
my|in
in|no
no|that
the|but
but|the
the|to
the|to
to|the
to|shall
the|shall
shall|and
the|and
shall|the
and|the
the|the
the|all
come|all
but|all
all|but
hamlet|but
all|hamlet
but|hamlet
but|and
hamlet|and
a|or
that|or
a|that
or|that
the|that
or|the
that|the
the|not
not|his
and|to
me|to
to|me
all|me
to|all
me|all
the|all
to|the
me|the
all|the
all|of
the|of
this|of
the|this
of|this
an|tis
tis|an
an|that
that|to
and|in
in|it
it|that
it|that
it|it
that|it
should|it
that|

to|his
it|it
my|it
it|my
my|and
and|you
in|you
and|in
you|in
the|in
you|the
in|the
in|of
the|of
your|of
the|your
of|your
of|out
your|out
of|out
your|of
out|of
the|of
out|the
of|the
of|and
the|and
and|of
of|the
the|is
is|if
if|her
her|to
the|to
her|the
to|the
not|the
the|the
the|of
the|of
the|the
of|the
their|be
be|and
in|and
be|in
and|in
the|in
and|the
in|the
in|and
the|and
and|of
most|are
are|most
are|be
most|be
be|then
in|to
as|to
my|to
as|my
to|my
to|but
my|but
good|but
my|good
but|good
my|good
but|my
good|my
good|do
my|do
not|do
my|not
do|not
as|not
do|as
not|as
some|as
do|some
not|some
as|some
some|do
do|me
the|me
do|the
me|the
me|and
the|and
and|to
to|like
a|like
like|a
like|and
a|and
the|of
of|and
and|not
his|not
and|his
not|his
his|enter
a|upon
upon|a
for|the
the|in
the|in
in|the
in|of
the|of
your|of
the|your
of|your
of|and
your|and
you|and
your|you
and|you
are|you
and|are
you|are
you|for
are|for
there|for
are|there
for|there
my|there
for|my
there|my
there|with
my|with
thee|wit

shall|in
the|in
shall|the
in|the
that|from
from|that
that|the
the|of
of|all
the|all
all|the
the|of
a|of
of|a
of|to
a|to
his|to
a|his
to|his
lord|my
my|lord
it|lord
my|it
lord|it
lord|enter
it|enter
enter|and
and|of
of|us
be|us
us|be
thou|be
us|thou
be|thou
a|thou
us|a
be|a
thou|a
thou|of
a|of
of|or
thee|with
with|thee
with|from
thee|from
from|or
or|from
from|be
thy|be
from|thy
be|thy
thy|or
or|thou
thou|in
such|in
thou|such
in|such
a|such
in|a
such|a
a|that
i|that
that|i
will|i
that|will
i|will
speak|will
that|speak
i|speak
will|speak
to|speak
i|to
will|to
speak|to
thee|to
will|thee
speak|thee
to|thee
i'll|thee
speak|i'll
to|i'll
thee|i'll
thee|thee
i'll|thee
hamlet|thee
i'll|hamlet
thee|hamlet
king|hamlet
thee|king
hamlet|king
father|king
thee|father
hamlet|father
king|father
let|me
me|let
me|let
me|me
let|me
not|me
me|not
let|not
me|not
me|in
not|in
in|but
but|why
thy|why
but|thy
why|thy
in|have
have|their
their|why
the|why
their|the
why|the
the|we
we|thee
thee|hath
hath|his
his|and


...

the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
b

are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|ar

coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
c

the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
bri

are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british
coming|are
the|are
british|are
coming|are
the|coming
british|coming
are|coming
the|coming
british|the
are|the
coming|the
british|the
are|british
coming|british
the|british
are|british


In [167]:
model.save('hamModel.h5')

In [168]:
from keras import backend as K
get_first = K.function(
    [model.layers[0].input],
    [model.layers[0].output])
get_first([[x_train[10]]])

[array([[0.78777325, 0.7494471 , 1.0767618 , 0.61824214, 0.36592788,
         1.4153857 , 1.4168702 , 0.7078819 , 0.3166803 , 0.39324832,
         0.6751762 , 0.33790252, 1.1173972 , 0.524128  , 0.7164112 ]],
       dtype=float32)]

In [169]:
def magic(word):
    return get_first([[wordToVector[word]]])[0]

In [172]:
dot = {word1 : {word2 : wordDot(word1, word2) for word2 in wordToVector.keys()} for word1 in wordToVector.keys()}

In [175]:
def wordDot(word1, word2):
    return magic(word1).dot(magic(word2).transpose())[0][0]

In [176]:
print(wordDot("hamlet", "lord"))
print(wordDot("hamlet", "the"))
print(wordDot("the","british"))
print(wordDot("the","in"))
print(wordDot("man","son"))
print(wordDot("a","the"))
print(wordDot("king","henry"))
print(wordDot("without", "clown"))

11.785418
7.202409
15.440334
6.8489676
12.207624
8.170089
10.635903
7.5380383


In [173]:
def associated(word):
    toSort = [(dot[word][otherWord], otherWord) for otherWord in wordToVector.keys()]
    sort = sorted(toSort, key=lambda x:x[0], reverse=False)
    return [tup[1] for tup in sort]

In [179]:
associated("enter")

['hast',
 'thy',
 'more',
 'is',
 'my',
 'thou',
 'dost',
 'no',
 'clown',
 'so',
 'art',
 'the',
 "i'll",
 'wilt',
 'hath',
 'too',
 'his',
 'that',
 'be',
 'how',
 'to',
 'she',
 'find',
 'doth',
 'why',
 'never',
 'a',
 'of',
 'me',
 'in',
 'nay',
 'mad',
 'cannot',
 'but',
 'again',
 'one',
 'thing',
 'nor',
 'else',
 'let',
 'our',
 'had',
 'an',
 'tis',
 'your',
 'been',
 'since',
 'second',
 'thine',
 'for',
 'not',
 'when',
 'should',
 'mine',
 'though',
 'none',
 'long',
 'said',
 'was',
 'well',
 'gone',
 'could',
 'or',
 'then',
 'first',
 'marry',
 'forth',
 'nothing',
 'her',
 'any',
 'now',
 'honour',
 'will',
 'there',
 'you',
 'us',
 'o',
 'set',
 'may',
 'young',
 'what',
 'yet',
 'without',
 'their',
 'thus',
 'till',
 'they',
 'hostess',
 'than',
 'down',
 'these',
 'might',
 'from',
 'like',
 'shall',
 'stand',
 'every',
 'did',
 'we',
 'done',
 'itself',
 'if',
 'yea',
 'them',
 'percy',
 'came',
 'leave',
 "there's",
 'here',
 'were',
 'horatio',
 'two',
 'who',
 

In [156]:
wordToVector.keys()

dict_keys(['the', 'and', 'of', 'to', 'a', 'i', 'you', 'are', 'my', 'in', 'that', 'coming', 'british', 'is', 'it', 'with', 'not', 'this', 'me', 'his', 'for', 'your', 'but', 'be', 'as', 'he', 'thou', 'so', 'have', 'him', 'what', 'lord', 'will', 'shall', 'we', 'our', 'by', 'all', 'on', 'do', 'no', 'if', 'thy', 'good', 'come', 'now', 'from', 'at', 'king', 'sir', 'or', 'thee', 'they', 'let', 'well', 'then', 'more', 'was', 'which', 'an', 'how', 'hath', 'like', 'upon', 'them', 'here', 'there', 'when', 'their', 'enter', 'us', 'would', 'did', "i'll", 'her', 'than', 'go', 'out', 'make', 'may', 'such', 'man', 'these', 'up', 'must', 'why', 'say', 'am', 'give', 'should', 'john', 'yet', 'doth', 'had', 'know', 'where', 'most', 'see', 'too', 'some', 'speak', 'were', 'tis', 'love', 'father', 'time', 'tell', 'one', 'first', 'very', 'who', 'god', 'prince', 'much', 'art', 'exeunt', 'hand', 'down', 'o', 'never', 'can', 'life', 'heaven', 'mine', 'blood', 'away', 'death', 'hear', 'peace', 'think', 'true', 's